In [1]:
!pip install keras-tuner

     |████████████████████████████████| 102kB 6.7MB/s 
  Created wheel for kt-legacy: filename=kt_legacy-1.0.3-cp37-none-any.whl size=9569 sha256=e4ac4e66464af7388d26a6138765f88876555ebed36c63d7439c6586df43491b
  Stored in directory: /root/.cache/pip/wheels/95/99/a8/6b116b0f69be60cc475d1cd36680f3a09f284d86655bb99d93
Successfully built kt-legacy


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.5.0


In [6]:
fashion_mnist = keras.datasets.fashion_mnist

In [8]:
(train_images,train_labels),(test_images,test_lables)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [9]:
train_images= train_images/255.0
test_images= test_images/255.0

In [10]:
train_images[0].shape

(28, 28)

In [11]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [12]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [13]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


# New Section

In [14]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [15]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 41s]
val_accuracy: 0.9138333201408386

Best val_accuracy So Far: 0.9138333201408386
Total elapsed time: 00h 03m 03s
INFO:tensorflow:Oracle triggered exit


In [20]:
model=tuner_search.get_best_models(num_models=1)[0]

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 64)        102464    
_________________________________________________________________
flatten (Flatten)            (None, 30976)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                2973792   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 3,077,866
Trainable params: 3,077,866
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1319 - accuracy: 0.9504 - val_loss: 0.2676 - val_accuracy: 0.9143
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0945 - accuracy: 0.9650 - val_loss: 0.3202 - val_accuracy: 0.9077
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0636 - accuracy: 0.9763 - val_loss: 0.3722 - val_accuracy: 0.9093
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0475 - accuracy: 0.9821 - val_loss: 0.4080 - val_accuracy: 0.9142
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0351 - accuracy: 0.9877 - val_loss: 0.4310 - val_accuracy: 0.9135
Epoch 9/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0295 - accuracy: 0.9891 - val_loss: 0.4925 - val_accuracy: 0.9080
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0255 - accuracy: 0.9909 - val_loss: 0.5542 - val_accuracy